In [ ]:
import os
import re
from datetime import datetime
import pandas as pd

In [ ]:
# Read in the CSV file
df = pd.read_csv(
    "./data/incoming_first_year_application/incoming_first_year_application.csv"  # Give a location to the csv data file, could be a real path or a relative path.
    # If you want to use the current path, follow below steps
    # 1. In the location of this python notebook (ipynb) file, create a folder "data".
    # 2. In data, create a folder "incoming_first_year_application" (*).
    # 3. Extract the zip file downloaded into (*).
)
print(len(df))
df.head(5)

In [ ]:
# Drop duplicate application, retain the latest one
print("Before:", len(df))
df['date'] = pd.to_datetime(df['Completed'])
df_sorted = df.sort_values(by=['Rhodes ID', 'date'], ascending=[True, False])
df = df_sorted.drop_duplicates(subset='Rhodes ID', keep='first')
print("After:", len(df))

In [ ]:
df.columns

In [ ]:
cols = ["Rhodes ID", "Completed", "Last Name", "First Name", "Preferred Name", "Zip Code", "Zip Code", "Possible Majors (or N/A)", "Upload Resume", "Duties" + "Do you have experience in customer service? If so, please list. , Computer Skills (hardware or software) , Other Education/Training/Skills" + "1. 2. 3. 4. 5. 1..1 2..1"]

In [ ]:
def is_us_zip_code(zip_code):
    """
    Returns:
        bool: True if it matches a US zip code format, False otherwise.
    """
    # Regex for 5-digit US zip code
    # ^\d{5}$ matches exactly 5 digits from start to end
    zip_code = str(zip_code)
    five_digit_pattern = re.compile(r"^\d{5}$")

    # Regex for ZIP+4 US zip code
    # ^\d{5}-\d{4}$ matches 5 digits, a hyphen, and 4 digits from start to end
    zip_plus_four_pattern = re.compile(r"^\d{5}-\d{4}$")

    if five_digit_pattern.match(zip_code):
        return True
    elif zip_plus_four_pattern.match(zip_code):
        return True
    else:
        return False

def get_skills(row):
    '''
    Merge questions in "skills" into 1 answer.
    '''
    skills = ["Duties", "Do you have experience in customer service? If so, please list.", "Computer Skills (hardware or software)", "Other Education/Training/Skills"]
    skills = [x for x in row[skills].tolist() if isinstance(x, str)]
    return " ".join(skills)

def get_pref(row):
    '''
    Get student placement preferences. Merge them by "#" so we can split in excel.
    '''
    cols = "1. 2. 3. 4. 5. 1..1 2..1".split()
    data = [x for i, x in enumerate(row[cols].tolist()) if isinstance(x, str)]
    return "#".join(data)

In [ ]:
# Process "Dom or Int", "Skills", "Preferences" columns
df['Dom or Int'] = df.apply(lambda x : "Domestic" if is_us_zip_code(x["Zip Code"]) else "International", axis=1)
df['Skills'] = df.apply(lambda x : get_skills(x) , axis=1)
df["Preferences"] = df.apply(lambda x : get_pref(x) , axis=1)

In [ ]:
# Set up final data
# Filter by wanted columns in cols
# Only
cols = ["Rhodes Email", "Rhodes ID", "Completed", "Last Name", "First Name", "Preferred Name", "Dom or Int", "Zip Code", "Possible Majors (or N/A)", "Upload Resume", "Skills", "Preferences"]
filtered_df = df[cols]
filtered_df.head(5)

In [ ]:
# Export final data to "./data/New_App.xlsx"
excel_file_name = "./data/New_App.xlsx"
if os.path.exists(excel_file_name):
    file_mode = 'a'
    writer_args = {'engine': 'openpyxl', 'mode': file_mode, 'if_sheet_exists': 'replace'}
else:
    file_mode = 'w'
    writer_args = {'engine': 'openpyxl', 'mode': file_mode}
    
with pd.ExcelWriter(excel_file_name, **writer_args) as writer:
    filtered_df.to_excel(writer, index=False)